In [41]:
import pandas as pd
import numpy as np
import json
v1=json.load(open('./MC1_dupmerge.json'))
nodes_df=pd.DataFrame(v1['nodes'])
links_df=pd.DataFrame(v1['links'])

merged_info=links_df.merge(nodes_df,how='left',left_on='source',right_on='id')
merged_info=merged_info.merge(nodes_df,how='left',left_on='target',right_on='id')
merged_info.drop(['id_x','id_y','dataset_y'],axis=1,inplace=True)
merged_info.rename(columns={'type_x':'link_type','type_y':'source_type','country_x':'source_country','country':'source_country',
                            'type':'target_type','country_y':'target_country'},inplace=True)
merged_info.info()
df = merged_info
# "Mar de la Vida OJSC",
# "979893388",
# "Oceanfront Oasis Inc Carriers",
# "8327"

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10747 entries, 0 to 10746
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   source          10747 non-null  object 
 1   target          10747 non-null  object 
 2   link_type       10747 non-null  object 
 3   weight          10746 non-null  float64
 4   source_type     8257 non-null   object 
 5   dataset_x       10747 non-null  object 
 6   source_country  2902 non-null   object 
 7   target_type     8079 non-null   object 
 8   target_country  3421 non-null   object 
dtypes: float64(1), object(8)
memory usage: 839.6+ KB


In [42]:
# 将数据按照source和target的大小排序
df[['source', 'target']] = np.sort(df[['source', 'target']], axis=1)

# 按照source、target和link_type进行分组，统计每组的数量
grouped = df.groupby(['source', 'target', 'link_type']).size().reset_index(name='count')

# 找到存在数量为2的组
duplicates = grouped[grouped['count'] == 2]

# 找到对应的边
result = pd.merge(df, duplicates, on=['source', 'target', 'link_type'], how='inner')

# 输出结果
print(f"存在{len(duplicates)}对方向相反但link_type相同的边：")
result

存在295对方向相反但link_type相同的边：


,source,target,link_type,weight,source_type,dataset_x,source_country,target_type,target_country,count
0,12744,Spanish Shrimp Carriers,ownership,0.900140,company,MC1,Nalakond,organization,NaN,2
1,12744,Spanish Shrimp Carriers,ownership,0.917011,organization,MC1,NaN,company,Nalakond,2
2,3506021,Spanish Shrimp Carriers,ownership,0.964213,company,MC1,Nalakond,organization,NaN,2
3,3506021,Spanish Shrimp Carriers,ownership,0.889810,organization,MC1,NaN,company,Nalakond,2
4,Spanish Shrimp Carriers,Turkish Sword Buoy Incorporated Forwading,membership,0.862103,company,MC1,Nalakond,company,Oceanus,2
...,...,...,...,...,...,...,...,...,...,...
585,Sea Harvest,Sea Products,membership,0.960678,NaN,MC1,NaN,NaN,NaN,2
586,Madhya Pradesh Market BV,Sea Harvest,family_relationship,0.838762,NaN,MC1,NaN,NaN,NaN,2
587,Madhya Pradesh Market BV,Sea Harvest,family_relationship,0.855869,NaN,MC1,NaN,NaN,NaN,2
588,Fresh On Time Seafood,Intrepid Navigators,membership,0.811744,NaN,MC1,NaN,NaN,NaN,2


In [43]:
# 定义需要筛选的节点列表

result_op = pd.DataFrame()
for row in result.iterrows():
    if row[1]['link_type'] == 'ownership' or row[1]['link_type'] == 'partnership':
        result_op = result_op.append(row[1])

result_op

/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/3353514927.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_op = result_op.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/3353514927.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_op = result_op.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/3353514927.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_op = result_op.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/3353514927.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_op = result_op

,source,target,link_type,weight,source_type,dataset_x,source_country,target_type,target_country,count
0,12744,Spanish Shrimp Carriers,ownership,0.900140,company,MC1,Nalakond,organization,NaN,2
1,12744,Spanish Shrimp Carriers,ownership,0.917011,organization,MC1,NaN,company,Nalakond,2
2,3506021,Spanish Shrimp Carriers,ownership,0.964213,company,MC1,Nalakond,organization,NaN,2
3,3506021,Spanish Shrimp Carriers,ownership,0.889810,organization,MC1,NaN,company,Nalakond,2
8,185040354,4,partnership,0.857406,organization,MC1,NaN,organization,NaN,2
...,...,...,...,...,...,...,...,...,...,...
575,Ocean Fisheries Llc,SeaSpray Wave SRL Solutions,partnership,0.849936,NaN,MC1,NaN,NaN,NaN,2
578,Ocean Fisheries Llc,SeaSpray Wave SRL Solutions,ownership,2.456995,NaN,MC1,NaN,NaN,NaN,2
579,Ocean Fisheries Llc,SeaSpray Wave SRL Solutions,ownership,6.033668,NaN,MC1,NaN,NaN,NaN,2
582,SMILE,âIllegal,partnership,0.944864,NaN,MC1,NaN,NaN,NaN,2


In [44]:
result_om = pd.DataFrame()
for row in result.iterrows():
    if row[1]['link_type'] == 'ownership' or row[1]['link_type'] == 'membership':
        result_om = result_om.append(row[1])

result_om

/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2353991764.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_om = result_om.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2353991764.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_om = result_om.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2353991764.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_om = result_om.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2353991764.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_om = result_om

,source,target,link_type,weight,source_type,dataset_x,source_country,target_type,target_country,count
0,12744,Spanish Shrimp Carriers,ownership,0.900140,company,MC1,Nalakond,organization,NaN,2
1,12744,Spanish Shrimp Carriers,ownership,0.917011,organization,MC1,NaN,company,Nalakond,2
2,3506021,Spanish Shrimp Carriers,ownership,0.964213,company,MC1,Nalakond,organization,NaN,2
3,3506021,Spanish Shrimp Carriers,ownership,0.889810,organization,MC1,NaN,company,Nalakond,2
4,Spanish Shrimp Carriers,Turkish Sword Buoy Incorporated Forwading,membership,0.862103,company,MC1,Nalakond,company,Oceanus,2
...,...,...,...,...,...,...,...,...,...,...
581,Ocean Fisheries Llc,Tamil Nadu S.p.A.,membership,0.833250,NaN,MC1,NaN,NaN,NaN,2
584,Sea Harvest,Sea Products,membership,0.904079,NaN,MC1,NaN,NaN,NaN,2
585,Sea Harvest,Sea Products,membership,0.960678,NaN,MC1,NaN,NaN,NaN,2
588,Fresh On Time Seafood,Intrepid Navigators,membership,0.811744,NaN,MC1,NaN,NaN,NaN,2


In [45]:
result_o = pd.DataFrame()
for row in result.iterrows():
    if row[1]['link_type'] == 'ownership':
        result_o = result_o.append(row[1])

result_o

/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/1922242207.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_o = result_o.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/1922242207.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_o = result_o.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/1922242207.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_o = result_o.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/1922242207.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_o = result_o.append(

,source,target,link_type,weight,source_type,dataset_x,source_country,target_type,target_country,count
0,12744,Spanish Shrimp Carriers,ownership,0.900140,company,MC1,Nalakond,organization,NaN,2
1,12744,Spanish Shrimp Carriers,ownership,0.917011,organization,MC1,NaN,company,Nalakond,2
2,3506021,Spanish Shrimp Carriers,ownership,0.964213,company,MC1,Nalakond,organization,NaN,2
3,3506021,Spanish Shrimp Carriers,ownership,0.889810,organization,MC1,NaN,company,Nalakond,2
10,185040354,3506021,ownership,0.841343,organization,MC1,NaN,organization,NaN,2
...,...,...,...,...,...,...,...,...,...,...
549,05d471b4-0e8c-4879-80b2-ef600220e09d,Texasâ,ownership,0.853588,location,MC1,NaN,NaN,NaN,2
558,Christopher Fields,Sonia Fisheries Pvt,ownership,0.905857,person,MC1,Country_470077,NaN,NaN,2
559,Christopher Fields,Sonia Fisheries Pvt,ownership,0.868182,NaN,MC1,NaN,person,Country_470077,2
578,Ocean Fisheries Llc,SeaSpray Wave SRL Solutions,ownership,2.456995,NaN,MC1,NaN,NaN,NaN,2


In [46]:
# 定义需要筛选的节点列表
target_nodes = ["Mar de la Vida OJSC", "979893388", "Oceanfront Oasis Inc Carriers", "8327", 'Spanish Shrimp  Carriers', 'Faroe Islands Shrimp Shark',  'b8567859-bf54-49fd-8332-5775e19c65af', 'Mar del Oeste Pic', 'SeaSpray Wave SRL Solutions', 'png xi  Line', '7505050', '435054320', '160']
cnt = 0
owner_cnt = 0
member_cnt = 0
family_cnt = 0
partner_cnt = 0
result_4points = pd.DataFrame()
for row in result.iterrows():
    if (row[1]['source'] in target_nodes) or (row[1]['target'] in target_nodes):
        cnt = cnt + 1
        result_4points = result_4points.append(row[1])
        if row[1]['link_type'] == 'ownership':
            owner_cnt = owner_cnt + 1
        elif row[1]['link_type'] == 'membership':
            member_cnt = member_cnt + 1
        elif row[1]['link_type'] == 'family_relationship':
            family_cnt = family_cnt + 1
        elif row[1]['link_type'] == 'partnership':
            partner_cnt = partner_cnt + 1

print(cnt/2)
print(owner_cnt/2)
print(member_cnt/2)
print(family_cnt/2)
print(partner_cnt/2)
result_4points

/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2180356102.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_4points = result_4points.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2180356102.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_4points = result_4points.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2180356102.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_4points = result_4points.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2180356102.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

76.0
17.0
23.0
18.0
18.0


/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2180356102.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_4points = result_4points.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2180356102.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_4points = result_4points.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2180356102.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_4points = result_4points.append(row[1])
/var/folders/rw/kvt1fn_n0h951yj9_wjsc5wm0000gn/T/ipykernel_14209/2180356102.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

,source,target,link_type,weight,source_type,dataset_x,source_country,target_type,target_country,count
0,12744,Spanish Shrimp Carriers,ownership,0.900140,company,MC1,Nalakond,organization,NaN,2
1,12744,Spanish Shrimp Carriers,ownership,0.917011,organization,MC1,NaN,company,Nalakond,2
2,3506021,Spanish Shrimp Carriers,ownership,0.964213,company,MC1,Nalakond,organization,NaN,2
3,3506021,Spanish Shrimp Carriers,ownership,0.889810,organization,MC1,NaN,company,Nalakond,2
4,Spanish Shrimp Carriers,Turkish Sword Buoy Incorporated Forwading,membership,0.862103,company,MC1,Nalakond,company,Oceanus,2
...,...,...,...,...,...,...,...,...,...,...
575,Ocean Fisheries Llc,SeaSpray Wave SRL Solutions,partnership,0.849936,NaN,MC1,NaN,NaN,NaN,2
576,Ocean Fisheries Llc,SeaSpray Wave SRL Solutions,membership,7.925211,NaN,MC1,NaN,NaN,NaN,2
577,Ocean Fisheries Llc,SeaSpray Wave SRL Solutions,membership,6.815232,NaN,MC1,NaN,NaN,NaN,2
578,Ocean Fisheries Llc,SeaSpray Wave SRL Solutions,ownership,2.456995,NaN,MC1,NaN,NaN,NaN,2
